# Zapytania złożone

Przed rozpoczęciem pracy konieczne jest skonfigurowanie bazy oraz załadowanie danych:
```ruby
$:.unshift "."
require 'db_setup'
require 'db_load'
```

In [6]:
$:.unshift "."
require 'db_setup'
require 'db_load'

false

W dalszych zadaniach będzie wykonywać polecenia korzystając z następujących klas, zmapowanych na odpowiadające im 
table w bazie danych:
```ruby
class Author < ActiveRecord::Base
  # name      (string)
  # surname   (string)
  # born      (datetime) 
  # died      (datetime)
  # image_url (string)
  
  has_many :books
end

class Book < ActiveRecord::Base
  # title     (string)
  # author    (Author)
  # published (integer)

  belongs_to :author
end
```

ActiveRecord udostępnia interfejs pozwalający wyszukiwać dane w bazie również z wykorzystaniem związków zdefiniowanych pomiędzy
klasami. Do definiowania tych związków służą polecenia `has_many`, `belongs_to`, `has_one` oraz `has_and_belongs_to_many`.
Przykładowo jeśli chcemy dowiedzieć się kto był autorem pierwszej książki wykonujemy zapytanie:
```ruby
book = Book.first
puts book.title
puts book.author
```

In [7]:
book = Book.first
puts book.title
puts book.author

Végjáték (regény)
#<Author:0x00000002a2d850>


Niestety prezentowany wynik nie jest zbyt użyteczny. Ponieważ jednak metoda `author` zwraca obiekt, możemy na nim wywoływać 
metody - tak samo jak w przypadku autora załadowanego bezpośrednio z wykorzystaniem metod zdefiniowanych dla klasy `Author`
```ruby
book = Book.first
puts book.title
puts "#{book.author.name} #{book.author.surname}"
```

In [8]:
book = Book.first
puts book.title
puts "#{book.author.name} #{book.author.surname}"

Végjáték (regény)
Orson Scott Card


Co więcej, jeśli mamy do czynienia ze związkiem jeden-do-wiele bądź wiele-do-wiele, możemy wywyływać wszystkie dotychczas poznane 
metody na kolekcji powiązanych obiektów. Istotnie ułatwia to wyszukiwanie potrzebnych danych:
```ruby
books = Book.where("title like 'A%'")
books.each do |book|
  puts book.title
end

author = Author.where(surname: "Tołstoj").first
books = author.books.where("title like 'A%'")
books.each do |book|
  puts book.title
end
```

In [10]:
books = Book.where("title like 'A%'")
books.each do |book|
  puts book.title
end

author = Author.where(surname: "Tołstoj").first
books = author.books.where("title like 'A%'")
books.each do |book|
  puts book.title
end

Anna Karenina
Azazel (powieść)
Anna Karenina


[#<Book id: 33, title: "Anna Karenina", language: nil, author_id: 30, published: 1877, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">]

Różnica pomiędzy tymi zapytaniami polega na tym, że w pierwszym przypadku szukamy książki, których tytuł zaczyna się na "A" 
w całym zbiorze książek, a w drugi przypadku wśród książek napisanych przez Tołstoja.

Aby umożliwić formatowanie wyników zapytania, IRuby udostępnia metodę `IRuby.html`, która pozwala na wyświetlanie kodu
HTML. Przykładowo
```ruby
IRuby.html "<h1>Test HTML-a</h1>"
```

In [11]:
IRuby.html "<h1>Test HTML-a</h1>"

"<h1>Test HTML-a</h1>"

Dzięki temu będziemy mogli formatować wyniki zapytań, a wszczególności wyświetlać obrazy przedstwiające autorów oraz książki. 
Należy jednak wziąć pod uwagę, że nie wszystkie książki oraz nie wszyscy autorzy posiadają odpowiadające im obrazki.
Aby wyświetlić imię, nazwisko oraz obraz pierwszego autora piszemy:
```ruby
author = Author.where("image_url is not null").first
IRuby.html "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
```

In [12]:
author = Author.where("image_url is not null").first
IRuby.html "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"

"<h1>Orson Scott Card</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/>"

Możemy również zdefniować funkcję, np. `display_author`, której zadaniem będzie wyświetlenie autora w tej postaci. 
Jedyny problem polega na tym, że funkcja `IRuby.html` może być wywoływana tylko raz. Dlatego jeśli chcemy wyświetlić kilku autorów,
musimy połączyć odpowiadające im kody HTML i wynik przekazać do funkcji `html`
```ruby
def display_author(author)
  "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
end

author_1 = Author.where("image_url is not null").first
author_2 = Author.where("image_url is not null").offset(1).first
document = display_author(author_1)
document += display_author(author_2)
IRuby.html document
```

In [13]:
def display_author(author)
  "<h1>#{author.name} #{author.surname}</h1><img src='#{author.image_url}'/>"
end

author_1 = Author.where("image_url is not null").first
author_2 = Author.where("image_url is not null").offset(1).first
document = display_author(author_1)
document += display_author(author_2)
IRuby.html document

"<h1>Orson Scott Card</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Orson_Scott_Card_at_BYU_Symposium_20080216_closeup.jpg?width=300'/><h1>Andrzej Sapkowski</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Sapkowski.jpg?width=300'/>"

## Zadania

### Zadanie 1

Wyświetl wszystkich autorów, którzy urodzili się przed rokiem 1900.

In [14]:
authors = Author.where(born: (Date.parse('0001-1-1')..Date.parse('1900-1-1')))
authors.each do |author|
  if author.born != nil && author.died != nil
    puts "#{author.name} #{author.surname} #{author.born.strftime("%d-%m-%Y")} #{author.died.strftime("%d-%m-%Y")}"
  end
end

Juliusz Słowacki 04-09-1809 03-04-1849
Henryk Sienkiewicz 05-05-1846 05-11-1916
Eliza Orzeszkowa 06-06-1841 23-05-1910
Adam Mickiewicz 24-12-1798 26-11-1855
Jack London 12-01-1876 22-11-1916
Mikołaj Kopernik 19-02-1473 24-05-1543
William Szekspir 23-04-1564 03-05-1616
John Ronald Reuel Tolkien 03-01-1892 02-09-1973
Aleksander Dumas (ojciec) 24-07-1802 05-12-1870
Tadeusz Dołęga-Mostowicz 10-08-1898 20-09-1939
Vladimir Nabokov 22-04-1899 02-07-1977
Adolf Hitler 20-04-1889 30-04-1945
Benedykt Chmielowski 20-03-1700 07-04-1763
Aldous Huxley 26-07-1894 22-11-1963
Honoré de Balzac 20-05-1799 18-08-1850
Franz Kafka 03-07-1883 03-06-1924
Howard Phillips Lovecraft 20-08-1890 15-03-1937
Lew Tołstoj 09-09-1828 20-11-1910
Ilja Ilf 15-10-1897 13-04-1937
Gabriela Zapolska 30-03-1857 17-12-1921
Clive Staples Lewis 29-11-1898 22-11-1963
Iwan Turgieniew 09-11-1818 03-09-1883
Lucy Maud Montgomery 30-11-1874 24-04-1942
Aleksander Majkowski 17-07-1876 10-02-1938


[#<Author id: 2, name: "Juliusz", surname: "Słowacki", born: "1809-09-04", died: "1849-04-03", image_url: nil>, #<Author id: 3, name: "Henryk", surname: "Sienkiewicz", born: "1846-05-05", died: "1916-11-05", image_url: nil>, #<Author id: 4, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-05-23", image_url: nil>, #<Author id: 5, name: "Adam", surname: "Mickiewicz", born: "1798-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 8, name: "Jack", surname: "London", born: "1876-01-12", died: "1916-11-22", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 12, name: "Mikołaj", surname: "Kopernik", born: "1473-02-19", died: "1543-05-24", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 13, name: "Maria Franciszka", surname: "Kozłowska", born: "1862-05-27", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 15, name: "William", surname: "Szekspir", born: "1564-

### Zadanie 2

Zmodyfikuj funkcję wyświetlającą autorów tak aby obejmowała ich datę urodzin i śmierci w formacie `dzień.miesiąc.rok`. Następnie wyświetl wszystkich autorów,
którzy zmarli w XX wieku.

In [15]:


authors = Author.where(died: (Date.parse('1901-1-1')..Date.parse('2000-12-31')))
authors.each do |author|
  if author.born != nil && author.died != nil
    puts "#{author.name} #{author.surname} #{author.born.strftime("%d-%m-%Y")} #{author.died.strftime("%d-%m-%Y")}"
  end
end

Henryk Sienkiewicz 05-05-1846 05-11-1916
Eliza Orzeszkowa 06-06-1841 23-05-1910
Jack London 12-01-1876 22-11-1916
Antoni Gołubiew 25-02-1907 27-06-1979
George Orwell 25-06-1903 21-01-1950
John Ronald Reuel Tolkien 03-01-1892 02-09-1973
Tadeusz Dołęga-Mostowicz 10-08-1898 20-09-1939
Vladimir Nabokov 22-04-1899 02-07-1977
Adolf Hitler 20-04-1889 30-04-1945
Aldous Huxley 26-07-1894 22-11-1963
Franz Kafka 03-07-1883 03-06-1924
Howard Phillips Lovecraft 20-08-1890 15-03-1937
Karol Olgierd Borchardt 25-03-1905 20-05-1986
Lew Tołstoj 09-09-1828 20-11-1910
Janusz Zajdel 15-08-1938 19-07-1985
Isaac Asimov 04-10-1919 06-04-1992
Ilja Ilf 15-10-1897 13-04-1937
Jewgienij Pietrow (pisarz) 13-12-1903 02-07-1942
Gabriela Zapolska 30-03-1857 17-12-1921
Clive Staples Lewis 29-11-1898 22-11-1963
Lucy Maud Montgomery 30-11-1874 24-04-1942
Alfred Szklarski 21-01-1912 09-04-1992
Aleksander Majkowski 17-07-1876 10-02-1938
Ian Fleming 28-05-1908 12-08-1964


[#<Author id: 3, name: "Henryk", surname: "Sienkiewicz", born: "1846-05-05", died: "1916-11-05", image_url: nil>, #<Author id: 4, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-05-23", image_url: nil>, #<Author id: 8, name: "Jack", surname: "London", born: "1876-01-12", died: "1916-11-22", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 11, name: "Antoni", surname: "Gołubiew", born: "1907-02-25", died: "1979-06-27", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 14, name: "George", surname: "Orwell", born: "1903-06-25", died: "1950-01-21", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 17, name: "John Ronald Reuel", surname: "Tolkien", born: "1892-01-03", died: "1973-09-02", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 19, name: "Tadeusz", surname: "Dołęga-Mostowicz", born: "1898-08-10", died: "1939-09-20", image_url: "http:

### Zadanie 3

Wyświetl 5 najmłodszych autorów, którzy posiadają przypisane zdjęcie.

In [16]:
authors = Author.where("image_url is not null")
authors = Author.order(:born).last(5)
authors.each do |author|
  puts "#{author.name} #{author.surname} #{author.born}"
end

Adam Zelga 1953-03-11
Boris Akunin 1956-05-20
Rafał A. Ziemkiewicz 1964-09-13
J. K. Rowling 1965-07-31
Andrzej Pilipiuk 1974-03-20


[#<Author id: 49, name: "Adam", surname: "Zelga", born: "1953-03-11", died: nil, image_url: nil>, #<Author id: 31, name: "Boris", surname: "Akunin", born: "1956-05-20", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 50, name: "Rafał A.", surname: "Ziemkiewicz", born: "1964-09-13", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 16, name: "J. K.", surname: "Rowling", born: "1965-07-31", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 41, name: "Andrzej", surname: "Pilipiuk", born: "1974-03-20", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">]

### Zadanie 4

Zdefiniuj funkcję `display_book`, która będzie zwracała fragment kodu HTML obejmujący tytuł książki, rok jej wydania oraz
przypisany obrazek.
Wyświetl najwcześniej wydaną książkę.

In [17]:
def display_book(book)
    "<h1>#{book.title} #{book.published}</h1><img src='#{book.author.image_url}'/>"
end
book = Book.where("published is not null and published != 0").order(:published).first
document = display_book(book)
IRuby.html document

"<h1>De revolutionibus orbium coelestium 1543</h1><img src='http://commons.wikimedia.org/wiki/Special:FilePath/Nikolaus_Kopernikus.jpg?width=300'/>"

### Zadanie 5

Wyświetl wszystkie książki, które nie mają przypisanego roku wydania.

In [18]:
books = Book.where("published is null or published == 0")
books.each do |book|
  p  "#{book.author.name} #{book.author.surname} #{book.title} #{book.published}"
end

"Orson Scott Card Végjáték (regény) "
"William Szekspir Hamlet "
"Iwan Turgieniew Ojcowie i dzieci 0"


[#<Book id: 1, title: "Végjáték (regény)", language: nil, author_id: 6, published: nil, image_url: nil>, #<Book id: 10, title: "Hamlet", language: nil, author_id: 15, published: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Book id: 65, title: "Ojcowie i dzieci", language: nil, author_id: 43, published: 0, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">]

### Zadanie 6

Zmodyfikuj funkcję `display_author` tak, aby wyświetlała również wszystkie książki napisane przez autora. Użyj znacznika `<ul>`.
Korzystając z tej funkcji wyświetl opis Stanisława Lema.

In [23]:
def display_author(author)
  author = Author.where(surname: author).first
  books = author.books
  html = "<h1>#{author.name} #{author.surname}</h1>"
  html << "<ul>"
  books.each do |book|
    html << "<li> #{book.title}</li>"
  end
  html << "</ul>"
end

document = display_author("Lem")
IRuby.html document

"<h1>Stanisław Lem</h1><ul><li> Bajki robotów</li><li> Solaris (powieść)</li><li> Bezsenność (zbiór opowiadań)</li><li> Głos Pana</li><li> Inwazja z Aldebarana (zbiór opowiadań)</li><li> Katar (powieść)</li><li> Księga robotów</li><li> Maska (zbiór dzieł)</li><li> Niezwyciężony i inne opowiadania</li><li> Noc księżycowa</li><li> Pamiętnik znaleziony w wannie</li><li> Polowanie (zbiór opowiadań)</li><li> Powtórka (zbiór dzieł)</li><li> Pożytek ze smoka</li><li> Rasa drapieżców. Teksty ostatnie</li><li> Sezam i inne opowiadania</li><li> Summa technologiae</li><li> Wizja lokalna (powieść)</li></ul>"

### Zadanie 7

Wśród książek napisanych przez Lema wyświetl wszystkie, które zawieraja w tytule słowo "zbiór".

In [24]:
def display_author(author)
  author = Author.where(surname: author).first
  books = author.books.where("title like '%zbiór%'")
  html = "<h1>#{author.name} #{author.surname}</h1>"
  html << "<ul>"
  books.each do |book|
    html << "<li> #{book.title}</li>"
  end
  html << "</ul>"
end

document = display_author("Lem")
IRuby.html document

"<h1>Stanisław Lem</h1><ul><li> Bezsenność (zbiór opowiadań)</li><li> Inwazja z Aldebarana (zbiór opowiadań)</li><li> Maska (zbiór dzieł)</li><li> Polowanie (zbiór opowiadań)</li><li> Powtórka (zbiór dzieł)</li></ul>"

### Zadanie 8

Wśród książek napisanych przez Lema wyświetl te, które zostały wydane w roku jego śmierci.

In [25]:
def display_author(author)
  author = Author.where(surname: author).first
  died = author.died.year
  books = author.books.where("published = ?", died)
  html = "<h1>#{author.name} #{author.surname}</h1>"
  html << "<ul>"
  books.each do |book|
    html << "<li> #{book.title}</li>"
  end
  html << "</ul>"
end

document = display_author("Lem")
IRuby.html document

"<h1>Stanisław Lem</h1><ul><li> Rasa drapieżców. Teksty ostatnie</li></ul>"

### Zadanie 9

Wyświetl wszystkie książki znajdujące się w bazie. Opis powinien zawierać tytuł, imię i nazwisko autora, rok wydania oraz 
obraz przedstawiający książkę (o ile istnieje).

In [27]:
def display_books()
  books = Book.all
  list = "<h1>Książki</h1><ul>"
  books.each do |book|
    list << "<li>"
    list << "#{book.title} #{book.published} - #{book.author.name} #{book.author.surname}"
    if (book.image_url != nil)
      list << "<img src='#{book.image_url}'/>"
    end
    list << "</li>"
  end
  list << "</ul>"
end 

document = display_books()
IRuby.html document
  

"<h1>Książki</h1><ul><li>Végjáték (regény)  - Orson Scott Card</li><li>Pani Jeziora (powieść) 1999 - Andrzej Sapkowski</li><li>Bellew Zawierucha 1912 - Jack London</li><li>Fałszywy trop (powieść) 1995 - Henning Mankell</li><li>Bajki robotów 1964 - Stanisław Lem</li><li>Bolesław Chrobry (powieść) 1947 - Antoni Gołubiew</li><li>De revolutionibus orbium coelestium 1543 - Mikołaj Kopernik</li><li>Dzieło Wielkiego Miłosierdzia 1922 - Maria Franciszka Kozłowska</li><li>Folwark zwierzęcy 1945 - George Orwell<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Animalism_flag.svg?width=300'/></li><li>Hamlet  - William Szekspir<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Hamlet_quarto_3rd.jpg?width=300'/></li><li>Harry Potter 1997 - J. K. Rowling<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Harry_Potter_wordmark.svg?width=300'/></li><li>Hobbit, czyli tam i z powrotem 1937 - John Ronald Reuel Tolkien</li><li>Hrabia Monte Christo 1844 - Aleksander Dumas (ojciec)<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Louis_Français-Dantès_sur_son_rocher.jpg?width=300'/></li><li>Kariera Nikodema Dyzmy 1932 - Tadeusz Dołęga-Mostowicz</li><li>Krzyżacy (powieść) 1900 - Henryk Sienkiewicz<img src='http://commons.wikimedia.org/wiki/Special:FilePath/PL_Henryk_Sienkiewicz-Krzyżacy_0005.jpeg?width=300'/></li><li>Lolita 1955 - Vladimir Nabokov</li><li>Mein Kampf 1925 - Adolf Hitler<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Landsberg.Jail.jpg?width=300'/></li><li>Nad Niemnem 1888 - Eliza Orzeszkowa<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Nad_Niemnem_-_manuscript.jpg?width=300'/></li><li>Niedokończone opowieści 1980 - Christopher Tolkien</li><li>Nowe Ateny 1745 - Benedykt Chmielowski<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Nowe_ateny1.jpg?width=300'/></li><li>Nowy wspaniały świat 1932 - Aldous Huxley</li><li>Ogniem i mieczem 1884 - Henryk Sienkiewicz<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Podbipieta.JPG?width=300'/></li><li>Ojciec Goriot 1835 - Honoré de Balzac<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Father_Goriot_by_H._Daumier_(1842).JPG?width=300'/></li><li>Proces (powieść) 1925 - Franz Kafka<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Kafka_Der_Prozess_1925.jpg?width=300'/></li><li>Psałterz Dawidów 1579 - Jan Kochanowski<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Kochanowski_-_Psałterz_Dawidów_(1579).jpg?width=300'/></li><li>Quo vadis 1896 - Henryk Sienkiewicz<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Rzym_popiersie_Sienkiewicza_w_kosciele_Domine_Quo_Vadis.JPG?width=300'/></li><li>Romeo i Julia 1597 - William Szekspir<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Romeo_and_juliet_brown.jpg?width=300'/></li><li>Silmarillion 1977 - John Ronald Reuel Tolkien<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Silmarrillion,_Just_under_the_Cover.jpg?width=300'/></li><li>Solaris (powieść) 1961 - Stanisław Lem<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Solaris,_various_editions_02.jpg?width=300'/></li><li>Władca Pierścieni 1954 - John Ronald Reuel Tolkien<img src='http://commons.wikimedia.org/wiki/Special:FilePath/Unico_Anello.png?width=300'/></li><li>Zew Cthulhu (opowiadanie) 1928 - Howard Phillips Lovecraft</li><li>Znaczy Kapitan 1960 - Karol Olgierd Borchardt</li><li>Anna Karenina 1877 - Lew Tołstoj<img src='http://commons.wikimedia.org/wiki/Special:FilePath/AnnaKareninaTitle.jpg?width=300'/></li><li>Azazel (powieść) 1998 - Boris Akunin</li><li>Bezsenność (zbiór opowiadań) 1971 - Stanisław Lem</li><li>Cała prawda o planecie Ksi 1983 - Janusz Zajdel</li><li>Dekorator 1998 - Boris Akunin</li><li>Diamentowa karoca 2003 - Boris Akunin</li><li>Dives in misericordia 1980 - Jan Paweł II<img src='http://commons.wikimedia.org/wiki/Special:FilePath/John_paul_2_coa.svg?width=300'/></li><li>Droga na molo w Wigan 1937 - George Orwell</

### Zadanie 10

Wyświetl wszystkich autorów znajdujących się w bazie wraz z opisem zawierającym ich zdjęcie, datę urodzin i śmierci oraz spis 
opublikowanych książek. Pamiętaj o odpowiednim sformatowaniu dat oraz o tym, że nie wszyscy autorzy mają przypisane wszystkie
informacje.

In [32]:
def display_authors()
  html = "<h1>Lista autorów</h1>"
  authors = Author.all
  authors.each do |author|
    html << "<h2>#{author.name} #{author.surname}</h2>"
    if (author.image_url != nil)
      html << " <img src='#{author.image_url}' />"
    end
    if author.books.empty?
      html << "<p>Brak książek dla tego autora</p>"
    else
      html << "<p>Lista książek</p>"
    end
    author.books.each do |book|
        html << "<p>#{book.title} #{book.published}</p>"
    end
  end
end
  
document = display_authors()
IRuby.html document

[#<Author id: 2, name: "Juliusz", surname: "Słowacki", born: "1809-09-04", died: "1849-04-03", image_url: nil>, #<Author id: 3, name: "Henryk", surname: "Sienkiewicz", born: "1846-05-05", died: "1916-11-05", image_url: nil>, #<Author id: 4, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-05-23", image_url: nil>, #<Author id: 5, name: "Adam", surname: "Mickiewicz", born: "1798-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 6, name: "Orson Scott", surname: "Card", born: "1951-08-24", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 7, name: "Andrzej", surname: "Sapkowski", born: "1948-06-21", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 8, name: "Jack", surname: "London", born: "1876-01-12", died: "1916-11-22", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 9, name: "Henning", surname: "Mankell", born: "1948-02-03", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 10, name: "Stanisław", surname: "Lem", born: "1921-09-12", died: "2006-03-27", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 11, name: "Antoni", surname: "Gołubiew", born: "1907-02-25", died: "1979-06-27", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 12, name: "Mikołaj", surname: "Kopernik", born: "1473-02-19", died: "1543-05-24", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 13, name: "Maria Franciszka", surname: "Kozłowska", born: "1862-05-27", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 14, name: "George", surname: "Orwell", born: "1903-06-25", died: "1950-01-21", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 15, name: "William", surname: "Szekspir", born: "1564-04-23", died: "1616-05-03", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 16, name: "J. K.", surname: "Rowling", born: "1965-07-31", died: nil, image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 17, name: "John Ronald Reuel", surname: "Tolkien", born: "1892-01-03", died: "1973-09-02", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 18, name: "Aleksander Dumas", surname: "(ojciec)", born: "1802-07-24", died: "1870-12-05", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 19, name: "Tadeusz", surname: "Dołęga-Mostowicz", born: "1898-08-10", died: "1939-09-20", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 20, name: "Vladimir", surname: "Nabokov", born: "1899-04-22", died: "1977-07-02", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 21, name: "Adolf", surname: "Hitler", born: "1889-04-20", died: "1945-04-30", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 22, name: "Christopher", surname: "Tolkien", born: "1924-11-21", died: nil, image_url: nil>, #<Author id: 23, name: "Benedykt", surname: "Chmielowski", born: "1700-03-20", died: "1763-04-07", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 24, name: "Aldous", surname: "Huxley", born: "1894-07-26", died: "1963-11-22", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 25, name: "Honoré de", surname: "Balzac", born: "1799-05-20", died: "1850-08-18", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 26, name: "Franz", surname: "Kafka", born: "1883-07-03", died: "1924-06-03", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 27, name: "Jan", surname: "Kochanowski", born: nil, died: "1584-08-22", image_url: "http://commons.wikimedia.org/wiki/Special:FilePath...">, #<Author id: 28, name: "Howard Phillips", surname: